In [15]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

from gensim.models import Word2Vec
from nltk.corpus import treebank, brown





from gensim.scripts.glove2word2vec import glove2word2vec
glove_input_file = 'data/glove/glove.6B.100d.txt'
word2vec_output_file = 'glove.6B.100d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)


(400000, 100)

In [2]:
nchannels = 1 # for word embeddings
n_hid_filt = 16 ## total guess

class _netD(nn.Module):
    def __init__(self, ngpu):
        super(_netD, self).__init__()
        self.main = nn.Sequential(
            # input is (nchannels) x 64 x 64
            # in, out, kernal, stride, padding
            nn.Conv2d(nchannels, n_hid_filt, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (n_hid_filt) x 32 x 32
            nn.Conv2d(n_hid_filt, n_hid_filt * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_hid_filt * 2),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (n_hid_filt*2) x 16 x 16
            nn.Conv2d(n_hid_filt * 2, n_hid_filt * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_hid_filt * 4),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (n_hid_filt*4) x 8 x 8
            nn.Conv2d(n_hid_filt * 4, n_hid_filt * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(n_hid_filt * 8),
            nn.LeakyReLU(0.2, inplace=True),
            
            # state size. (n_hid_filt*8) x 4 x 4
            nn.Conv2d(n_hid_filt * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        if isinstanchannelse(input.data, torch.cuda.FloatTensor) and self.ngpu > 1:
            output = nn.parallel.data_parallel(self.main, input, range(self.ngpu))
        else:
            output = self.main(input)

        return output.view(-1, 1).squeeze(1)

In [12]:
b = Word2Vec(brown.sents())
# mr = Word2Vec(movie_reviews.sents())
t = Word2Vec(treebank.sents())

In [14]:
# sentence -> word2vec -> make 2D
print(b.most_similar('company', topn=5))
print(t.most_similar('company', topn=5))



[('Giants', 0.9529365301132202), ('appeal', 0.9520005583763123), ('secret', 0.940590500831604), ('feather', 0.9396657943725586), ('banter', 0.9371280670166016)]
[('is', 0.9996277689933777), ('that', 0.9996199011802673), ('they', 0.999491810798645), ('*T*-3', 0.9993422627449036), ('have', 0.9992862939834595)]


In [9]:
x = list(treebank.sents())

In [10]:
len(x)

3914

In [16]:
from gensim.models import KeyedVectors
# load the Stanford GloVe model
filename = 'glove.6B.100d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)
# calculate: (king - man) + woman = ?
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

[('queen', 0.7698541283607483)]


In [21]:
result = model.most_similar(positive=['dark', 'light'], negative=['white'], topn=5)
print(result)

[('lighter', 0.6182740926742554), ('darkness', 0.6158767938613892), ('bright', 0.6146994829177856), ('darker', 0.609158992767334), ('sunlight', 0.6041673421859741)]
